<a href="https://colab.research.google.com/github/amgito1648/clase-inteligencia-artificial/blob/main/Fundamento_Cuaderno_19_DBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color="blue"> Cuaderno 19. DBSCAN: Clustering basado en densidad

Introducción </font>

El algoritmo DBSCAN (Density-Based Spatial Clustering of Applications with Noise) identifica agrupaciones de puntos de datos en función de su densidad en el espacio. Este enfoque sigue la intuición humana al identificar regiones de alta densidad de observaciones, separadas por áreas de baja densidad. A diferencia de otros métodos como K-means, que requieren un número predefinido de clusters y asumen formas esféricas o convexas, DBSCAN es capaz de identificar agrupaciones con formas arbitrarias y es menos sensible a la presencia de ruido o outliers.

##Conceptos clave en DBSCAN

Vecindad 𝜀 (epsilon): Es el radio alrededor de un punto que define su vecindad. Este parámetro es clave para determinar qué puntos deben agruparse juntos. Se refiere al vecindario
ε-neighborhood de un punto.
$$ N_{\epsilon}(p) = \{ q \in D \mid d(p, q) \leq \epsilon \} $$


Número mínimo de puntos (min_samples): Es el número mínimo de puntos necesarios para que un área se considere densa y un cluster sea válido. Este parámetro ayuda a evitar que puntos aislados sean clasificados erróneamente como parte de un cluster.
$$|N_\varepsilon(p)| \geq \text{min_samples}$$

##Tipos de puntos:

**Punto central (Core point)**: Un punto es un "core point" si tiene al menos min_samples puntos en su vecindad 𝜀.

**Punto de frontera (Border point)**: Un punto que no es un core point, pero está dentro del 𝜀-vecindario de un core point.

**Ruido u outlier (Noise)**: Un punto que no es ni core point ni border point. Se considera ruido.

En la siguiente imagen de dos carateristicas (x,y) ##se observan los tipos de puntos, el naranja es ruido, el amarillo es punto central y el resto son puntos de frontera

![imagen](https://github.com/adiacla/bigdata/blob/master/dbscan.png?raw=true)

Tomada de: Pandey, P. (2020, October 22). DBSCAN Clustering. Machine Learning Geek. https://machinelearninggeek.com/dbscan-clustering/

##Conectividad entre puntos:

- Directamente alcanzable (directly density reachable): Un punto A es directamente alcanzable desde otro punto B si A está dentro del 𝜀-vecindario de B y B es un core point.

- Alcanzable (density reachable): Un punto A es alcanzable desde otro punto B si existe una secuencia de core points que conectan ambos puntos.

- Conectados densamente (density connected): Dos puntos A y B están densamente conectados si existe un core point C tal que A y B son alcanzables desde C.


##Algoritmo DBSCAN

El algoritmo de DBSCAN sigue estos pasos:

Para cada punto \( p \) no visitado:
   - Calcular su vecindad $N_{\epsilon}(p)$.
   - Si $|N_{\epsilon}(p)| \geq \text{min}_{samples}$, iniciar un nuevo cluster y expandirlo.
   - Marcar los puntos visitados y asignarlos al cluster.
   - Si $|N_{\epsilon}(p)| < \text{min}_{samples}$, marcarlo como ruido.

2. Expandir el cluster añadiendo los puntos frontera hasta que no se puedan agregar más.


Para cada core point no asignado a un cluster, se crea un nuevo cluster. Luego, se identifican y asignan al mismo cluster todos los puntos que están densamente conectados con este core point.

Repetir el proceso para todos los puntos que no han sido visitados. Aquellos que no pertenezcan a ningún cluster serán considerados outliers.

El resultado final es un conjunto de clusters, donde todos los puntos dentro de un cluster están densamente conectados entre sí. Además, si un punto A es densamente alcanzable desde cualquier otro punto dentro de un cluster, se considera que A pertenece al mismo cluster.


##Ventajas y desventajas de DBSCAN

**Ventajas:**

- No requiere que el usuario especifique el número de clusters de antemano.
- Es independiente de la forma de los clusters. Puede identificar clusters de forma arbitraria.
- Identifica automáticamente los outliers o puntos ruidosos, lo que asegura que los clusters no estén influenciados por datos atípicos.

**Desventajas:**

- Es determinístico solo si los datos se procesan en un orden específico. Los puntos de frontera que pueden pertenecer a más de un cluster dependen del orden en que se procesan.
- Si los clusters tienen densidades muy diferentes, el algoritmo puede no encontrar un valor adecuado de 𝜀
y $min_{samples}$ que funcione bien para todos los clusters.



#Taller Clustering de Clientes con modelo jerarquico

Vamos a realizar el mismo taller de clusterizacion de las secciones anteriores usando DBSCAN y vamos crear los clusteres.

En este paso preparamos el entorno de trabajo al importar bibliotecas clave y cargar un conjunto de datos sobre clientes. Esto permite realizar análisis exploratorios, preprocesamiento y clustering en pasos posteriores.

In [ ]:
# Importar bibliotecas
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cargar datos
url = "https://raw.githubusercontent.com/adiacla/bigdata/refs/heads/master/Shopping_CustomerData.csv"
data = pd.read_csv(url)

data

,ID,Genero,Edad,Ciudad,Ingresos,Credito,gastos,trabajo
0,1001,Male,49,Bengaluru,527547.58850,653,78,1
1,1002,Male,59,Bengaluru,207143.19760,630,63,1
2,1003,Female,54,Delhi,164423.84570,555,69,4
3,1004,Female,42,Bengaluru,56220.36443,699,30,1
4,1005,Female,30,Bengaluru,256194.36190,793,6,1
...,...,...,...,...,...,...,...,...
195,1196,Female,54,Delhi,317466.42070,601,52,4
196,1197,Female,20,Bengaluru,323305.50020,554,58,1
197,1198,Male,44,Chennai,109058.54430,844,36,2
198,1199,Male,28,Delhi,160116.89300,837,24,4


In [ ]:
#Verificamos nulos
data.isnull().sum()

,0
ID,0
Genero,0
Edad,0
Ciudad,0
Ingresos,0
Credito,0
gastos,0
trabajo,0


In [ ]:
data.dtypes

,0
ID,int64
Genero,object
Edad,int64
Ciudad,object
Ingresos,float64
Credito,int64
gastos,int64
trabajo,int64


##Codificar la variable 'Genero' (Male = 0, Female = 1):

Este paso transforma la información categórica en una forma numérica para su compatibilidad con algoritmos de clustering y selecciona las características clave que representan a cada cliente, preparándolas para el análisis posterior.

In [ ]:
# Preprocesamiento: Codificar 'Genero' y seleccionar columnas relevantes
data['Genero'] = data['Genero'].map({'Male': 0, 'Female': 1})

data_numeric = data[['Edad', 'Ingresos', 'Credito', 'gastos', 'Genero']]


##Estandarizar las variables

Este paso estandariza las características seleccionadas para garantizar que estén en la misma escala. Esto es esencial en métodos de clustering como DBSCAN, que dependen de las distancias entre puntos y pueden verse afectados por diferencias en las magnitudes de las variables.

In [ ]:
# Escalar los datos
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_numeric)
data_scaled= pd.DataFrame(data_scaled, columns=data_numeric.columns)
data_scaled

,Edad,Ingresos,Credito,gastos,Genero
0,0.216509,1.435993,-0.642037,0.952679,-1.128152
1,0.838660,-0.462074,-0.862520,0.429133,-1.128152
2,0.527584,-0.715143,-1.581486,0.638551,0.886405
3,-0.218997,-1.356137,-0.201071,-0.722668,0.886405
4,-0.965579,-0.171497,0.700033,-1.560341,0.886405
...,...,...,...,...,...
195,0.527584,0.191478,-1.140520,0.045199,0.886405
196,-1.587730,0.226068,-1.591072,0.254618,0.886405
197,-0.094567,-1.043125,1.188930,-0.513249,-1.128152
198,-1.090009,-0.740657,1.121827,-0.932086,-1.128152


##Reducir la dimensionalidad

Aunque este paso no  hemos revisado el tema PCA o Análisis de componentes principales,  que se va a trata en el siguiente cuaderno, para este ejercicio vamos reducir la dimensionalidad de los datos para facilitar el análisis y mejorar el rendimiento de algoritmos que son sensibles al número de dimensiones. DBSCAN es uno de ellos.

Este paso de reducir la dimensionalidad es casi obligatorio para todos los modelos de machine learning, paa generar nuevos modelo y compararlos con aquuellos que fueron creados con las columnas de la vista minable.

PCA identifica las combinaciones lineales de características originales que capturan la mayor parte de la variación en los datos, garantizando una representación más compacta sin perder información relevante.

In [ ]:
# Realizar PCA o reducción de dimensiones si es necesario
from sklearn.decomposition import PCA
pca = PCA(n_components=3)  # Por ejemplo, reducir a 3 componentes
data_reduced = pca.fit_transform(data_scaled)

data_reduced
columnas=['Componente1','Componente2','Componente3']
data_reduced=pd.DataFrame(data_reduced,columns=columnas)
data_reduced

,Componente1,Componente2,Componente3
0,0.237819,-1.244044,1.082193
1,-1.212485,-1.125587,0.056750
2,-0.085836,-1.649147,-0.758166
3,-0.401965,0.444681,-1.214713
4,0.279244,1.703739,-0.484185
...,...,...,...
195,0.152213,-1.066304,0.084188
196,1.118617,-0.946805,-1.298397
197,-1.088161,1.313741,-0.360947
198,-0.709640,1.738015,-0.707070


In [ ]:
from collections import Counter

best_result = {'eps': None, 'min_samples': None, 'score': -1, 'labels': None}

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X)
        mask = labels != -1
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        pct_noise = n_noise / len(labels)

        if n_clusters > 1 and len(set(labels[mask])) > 1 and pct_noise < 0.4:
            score = silhouette_score(X[mask], labels[mask])
            if score > best_result['score']:
                best_result = {
                    'eps': eps,
                    'min_samples': min_samples,
                    'score': score,
                    'labels': labels
                }

# Mostrar resultado óptimo
if best_result['labels'] is not None:
    print(f"Mejor resultado: eps={best_result['eps']}, min_samples={best_result['min_samples']}, silhouette={best_result['score']:.3f}")
    print(Counter(best_result['labels']))
else:
    print("No se encontró un clustering válido con múltiples clústeres y bajo ruido.")


Mejor resultado: eps=0.8499999999999999, min_samples=3, silhouette=0.233
Counter({np.int64(0): 189, np.int64(-1): 8, np.int64(1): 3})


## Ahora si vamos a hacer un modelo con DBSCAN

###Configrar los hiperparámetros



En este paso, se están configurando los hiperparámetros para el algoritmo DBSCAN:

**epsilon (eps):** Define el radio de proximidad para los puntos vecinos. En este caso, está configurado como 0.5, lo que significa que solo los puntos dentro de este radio se considerarán vecinos.

**min_samples:** Especifica el número mínimo de puntos en la vecindad para que un punto sea considerado como un "punto central". Está configurado en 10.

Luego, se aplica DBSCAN a los datos estandarizados con dbscan.fit_predict(data_scaled), que realiza el clustering y asigna etiquetas a cada punto de datos.

In [ ]:
import plotly.express as px
import pandas as pd

# Asegúrate de que data_reduced sea un DataFrame
if not isinstance(data_reduced, pd.DataFrame):
    data_reduced = pd.DataFrame(data_reduced, columns=['Componente1', 'Componente2', 'Componente3'])

# Añadir etiquetas del mejor resultado
data_reduced['Cluster'] = best_result['labels']

# Filtrar ruido si lo deseas (opcional)
# data_reduced = data_reduced[data_reduced['Cluster'] != -1]

# Gráfico interactivo
fig = px.scatter_3d(
    data_reduced,
    x='Componente1',
    y='Componente2',
    z='Componente3',
    color=data_reduced['Cluster'].astype(str),
    title=f"Clustering DBSCAN (eps={best_result['eps']}, min_samples={best_result['min_samples']})",
    labels={'color': 'Cluster'}
)

fig.update_layout(legend_title_text='Cluster')
fig.show()


In [ ]:
# Aplicar DBSCAN
labels

array([ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,
        0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
        0,  0,  0,  0,  0, -1,  0,  2,  0,  0,  0,  0,  0])

En el paso aplica el algoritmo DBSCAN para identificar clústeres en los datos reducidos. DBSCAN encuentra agrupaciones basándose en densidades locales, identificando regiones densas de puntos y clasificando puntos escasos como ruido. Las etiquetas obtenidas (almacenadas en clusters) se usarán para análisis y visualización.

**El número -1:**
En DBSCAN, el valor -1 identifica puntos clasificados como ruido. Esto ocurre cuando un punto no pertenece a ningún clúster porque:


* Vecinos insuficientes: El número de puntos dentro del radio epsilon no alcanza el umbral min_samples.

* Densidad aislada: Los puntos están demasiado dispersos y no forman parte de ninguna región densa.


In [ ]:
# cardinalidad de la variable clusters

print(np.unique(labels))

[-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]


**Cantidad de clústeres:**
En el resultado proporcionado, los clústeres únicos (excluyendo -1) son:

Esto indica que se detectaron 14 clústeres válidos.

##Asignar los clientes a los clústeres

In [ ]:
#Asignar el cluster a cada cliente
data['Cluster'] = labels
data

,ID,Genero,Edad,Ciudad,Ingresos,Credito,gastos,trabajo,Cluster
0,1001,0,49,Bengaluru,527547.58850,653,78,1,0
1,1002,0,59,Bengaluru,207143.19760,630,63,1,1
2,1003,1,54,Delhi,164423.84570,555,69,4,2
3,1004,1,42,Bengaluru,56220.36443,699,30,1,2
4,1005,1,30,Bengaluru,256194.36190,793,6,1,2
...,...,...,...,...,...,...,...,...,...
195,1196,1,54,Delhi,317466.42070,601,52,4,2
196,1197,1,20,Bengaluru,323305.50020,554,58,1,2
197,1198,0,44,Chennai,109058.54430,844,36,2,2
198,1199,0,28,Delhi,160116.89300,837,24,4,2


In [ ]:
# la frecuencia de clusters

# Obtener la frecuencia de cada cluster
frecuencia_clusters = data['Cluster'].value_counts()
print("Frecuencia de los clusters:")
frecuencia_clusters

Frecuencia de los clusters:


,count
Cluster,
2,127
-1,31
5,7
7,5
9,4
10,3
12,3
0,3
14,3


El cuadro anterior nos demuesra que hubo una mala distribución de los clusters

In [ ]:
dbscan = DBSCAN(eps=0.57, min_samples=2)
clusters = dbscan.fit_predict(data_reduced)

mask = clusters != -1
if len(set(clusters[mask])) > 1:
    silhouette_valid = silhouette_score(data_reduced[mask], clusters[mask])
    print(f"Silhouette Score sin ruido (eps=0.57, min_samples=2): {silhouette_valid}")
else:
    print("No hay suficientes clusters válidos para calcular el silhouette score.")

Silhouette Score sin ruido (eps=0.57, min_samples=2): -0.1435555130904698


Se concluye que con un epsilon de 0.9 y un min_samples de 6 se logra alcanzar el máximo Silhouette Score de 0.24

In [ ]:
# crear gráfico 3d de plotly para graficar los componentes principales de data_reduced con el clusters

import plotly.graph_objects as go
# Crear el gráfico 3D interactivo con Plotly
fig = go.Figure(data=[go.Scatter3d(
    x=data_reduced['Componente1'],
    y=data_reduced['Componente2'],
    z=data_reduced['Componente3'],
    mode='markers',
    marker=dict(
        size=5,
        color=data['Cluster'],  # Colorear por cluster
        colorscale='Viridis', # Escala de color
        opacity=0.8
    )
)])

# Configurar el layout del gráfico
fig.update_layout(
    title='Clusters de Clientes en 3D (PCA)',
    scene = dict(
        xaxis_title='Componente Principal 1',
        yaxis_title='Componente Principal 2',
        zaxis_title='Componente Principal 3'),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Mostrar el gráfico
fig.show()


 #Numero de clusteres y ruido
 calcula el número de clusters encontrados por DBSCAN y el número de observaciones consideradas "outliers" (es decir, aquellas que no se asignan a ningún cluster, representadas por el valor -1 en el resultado de fit_predict). La variable n_clusters cuenta la cantidad de clusters distintos (excluyendo los outliers), mientras que n_noise cuenta las observaciones que fueron etiquetadas como ruido (-1).

In [ ]:
# Número de clusters y observaciones "outliers"
n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
n_noise    = list(clusters).count(-1)

print(f'Número de clusters encontrados: {n_clusters}')
print(f'Número de outliers encontrados: {n_noise}')

Número de clusters encontrados: 15
Número de outliers encontrados: 31


In [ ]:
#persistencia al modelo final
import joblib as job
job.dump(dbscan,'dbscan.bin')


['dbscan.bin']